In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
DATA_PATH = Path(".") / "../../data"
RESULTS_PATH = Path(".") / "../../results"
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

In [3]:
therapeutics_ground_truth_path = DATA_PATH / "therapeutics" / "therapeutics_ground_truth_human_new.csv"
regression_results_path = RESULTS_PATH / "therapeutics" / "therapeutics_regression_results.csv"

In [4]:
import pandas as pd
# Read the CSV and save as FASTA
therapeutics_input_path = DATA_PATH / "therapeutics" / "therapeutics_gt_aa.fasta"

df = pd.read_csv(therapeutics_ground_truth_path)[['sequence_header', 'sequence_aa']]

# Save as FASTA file
with open(therapeutics_input_path, 'w') as f:
    for _, row in df.iterrows():
        f.write(f">{row['sequence_header']}\n")
        f.write(f"{row['sequence_aa']}\n")

In [5]:
from riot_na.api.api_mp import run_on_file_mp
from riot_na.api.api_mp import GENE_DB_DIR, InputType, Organism

run_on_file_mp(
    db_dir=GENE_DB_DIR,
    input_fasta_path=therapeutics_input_path,
    result_path=regression_results_path,
    input_type=InputType.AA,
    allowed_species=[Organism.HOMO_SAPIENS],
    extend_alignment=False,
)

100%|██████████| 1759/1759 [00:00<00:00, 1974.58it/s]


In [6]:
from notebooks.utils import base64_decode_series, validation_flags_comparison
import pandas as pd

new_df = pd.read_csv(regression_results_path, index_col=0, engine="pyarrow")
print(len(new_df))
new_df = base64_decode_series(new_df, "additional_validation_flags")
print(len(new_df))

old_df = pd.read_csv(therapeutics_ground_truth_path, index_col=0, engine="pyarrow") 
old_df = base64_decode_series(old_df, "additional_validation_flags")
print(len(old_df))

validation_flags_comparison(old_df, new_df)

3517


Error: Invalid base64-encoded string: number of data characters (25) cannot be 1 more than a multiple of 4

In [10]:
comparison_df = new_df[["v_call"]].join(old_df[["v_call"]], how="left", lsuffix="_new", rsuffix="_old")
comparison_df['same_v_call'] = comparison_df['v_call_new'] == comparison_df['v_call_old']
print(len(comparison_df.query("same_v_call == False")))
comparison_df.query("same_v_call == False")


43


,v_call_new,v_call_old,same_v_call
sequence_header,,,
Dezamizumab_heavy,IGHV1-3*01,IGHV1-69*06,False
Tomuzotuximab_heavy,IGHV3-33*01,IGHV4-4*08,False
Mipasetamab_heavy,IGHV3-30*03,IGHV3-NL1*01,False
Lanadelumab_heavy,IGHV3-23*01,IGHV3-23*03,False
Luveltamab_heavy,IGHV3-23*04,IGHV3-66*01,False
Lupartumab_heavy,IGHV3-11*01,IGHV3-48*03,False
Satumomab_heavy,IGHV1-69-2*01,IGHV1-3*01,False
Emicizumab_heavy,IGHV1-2*06,IGHV1-46*01,False
Pimurutamab_heavy,IGHV3-20*04,IGHV3-NL1*01,False


In [26]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

old_df.loc[["Tomuzotuximab_heavy"]].T.head(100).style.set_properties(**{'text-align': 'left'})

sequence_header,Tomuzotuximab_heavy
sequence_aa,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLEWLGVIWSGGNTDYNTPFTSRLSINKDNSKSQVFFKMNSLQSNDTAIYYCARALTYYDYEFAYWGQGTLVTVST
numbering_scheme,imgt
locus,igh
stop_codon,False
productive,True
complete_vdj,True
v_call,IGHV4-4*08
j_call,IGHJ4*02
germline_alignment_aa,QVQLQESGPGLVKPSETLSLTCTVSGGSISSYYWSWIRQPPGKGLEWIGYIYTSGSTNYNPSLKSRVTISVDTSKNQFSLKLSSVTAADTAVYYCARNNNNNNNNFDYWGQGTLVTVSS
sequence_alignment_aa,QVQLKQSGPGLVQPSQSLSITCTVSGFSLTNYGVHWVRQSPGKGLEWLGVIWSGGNTDYNTPFTSRLSINKDNSKSQVFFKMNSLQSNDTAIYYCARALTYYDYEFAYWGQGTLVTVST


In [13]:
new_df.loc[["Licaminlimab_heavy"]].T.head(100).style.set_properties(**{'text-align': 'left'})

sequence_header,Licaminlimab_heavy
sequence_aa,EVQLVESGGGSVQPGGSLRLSCTASGFTISRSYWICWVRQAPGKGLEWVGCIYGDNDITPLYANWAKGRFTISRDTSKNTVYLQMNSLRAEDTATYYCARLGYADYAYDLWGQGTTVTVSS
query_sequence_aa,EVQLVESGGGSVQPGGSLRLSCTASGFTISRSYWICWVRQAPGKGLEWVGCIYGDNDITPLYANWAKGRFTISRDTSKNTVYLQMNSLRAEDTATYYCARLGYADYAYDLWGQGTTVTVSS
numbering_scheme,imgt
locus,igh
stop_codon,False
productive,True
complete_vdj,True
v_call,IGHV3-74*01
j_call,IGHJ6*02
c_call,nan
